In [26]:
import numpy as np
import pandas as pd
import math
from scipy.stats import poisson
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import plotly.express as px


In [27]:
def sqrt_func(i, j):
    time.sleep(1)
    return i + j


Parallel(n_jobs=2)(delayed(sqrt_func)(i, 5)
                   for i in range(5))


[5, 6, 7, 8, 9]

## Functions

In [28]:
I = 48  # Number of intervals
d = 5  # length of interval
x = np.random.choice(5, I)
N = sum(x)
precision = 0.9999
beta = 9  # Average service time for a patient
no_show = 0 # Fraction of scheduled patients not showing up
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

print(N)

89


In [29]:
def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))

In [30]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))


def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [31]:
# Distribution to calculate service time of patients
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
def calculate_p(beta, size, precision=0.9999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

##########
## TEST ##
##########

probS = calcExponentialLimit(beta*N)+1
print(probS)
calculate_p(beta, probS, precision)


915


([0.00012340980408667956,
  0.0011106882367801166,
  0.004998097065510523,
  0.014994291196531574,
  0.033737155192196056,
  0.06072687934595293,
  0.09109031901892926,
  0.1171161244529091,
  0.13175564000952278,
  0.13175564000952278,
  0.11858007600857066,
  0.09702006218883041,
  0.0727650466416229,
  0.050375801521123684,
  0.03238444383500792,
  0.01943066630100472,
  0.010929749794315179,
  0.005786338126402172,
  0.0028931690632010803,
  0.0013704485036215655,
  0.0006167018266297063,
  0.0002643007828413016,
  0.00010812304752598687,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [32]:
def calculate_v(p, beta, precision, precision_limit, N, d, no_show=0):
  count = precision_limit
  limit = calcExponentialLimit(beta*N) + 1
  v = np.zeros((N+1, limit+d))
  add_v = np.zeros((N+1, limit+d))

  add_v[0][0] = 1
  for k in range(1, N+1):
    limit = calcExponentialLimit(beta*k)
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while z <= count:
        add_v[k][i] += p[z] * add_v[k-1][i-z]
        z += 1
      sum_v += add_v[k][i]
      i += 1

  for k in range(N+1):
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k+1):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  return v, limit

##########
## TEST ##
##########

p, precision_limit = calculate_p(beta, probS, precision)
calculate_v(p, beta, precision, precision_limit, N, d, no_show)


(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.52299797e-08, 2.74139635e-07, 2.46725672e-06, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.52648902e-08, 2.66874541e-08, 1.21878115e-08, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 914)

In [33]:
def calculateProbabilities(x, precision, limit, v, I, d):
  p_plus = np.zeros((I+1, limit+d+1))
  p_min = np.zeros((I+1, limit+d+1))

  # Constraint 1
  p_min[0][0] = 1

  # Constraint 2
  sum_p = 0
  i = 0
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I+1):  # calculate p_min and p_plus iteratively
    # Constraint 3
    for k in range(d+1):
      # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
      p_min[t][0] += p_plus[t-1][k]
    # Constraint 4
    for i in range(1, limit+1):

      # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
      p_min[t][i] = p_plus[t-1][i+d]

    # Constraint 5
    if t != I:  # I or I+1
      for i in range(limit+1):
        for j in range(i+1):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  return p_min, limit

##########
## TEST ##
##########

v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)
p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)


In [34]:
def calcTardiness(p_min, limit, I):
  tardiness = 0
  # print(p_min[I])
  for k in range(limit):
    tardiness += k * p_min[I][k]  # I+1
  return tardiness


def calcIdletime(I, d, tardiness, N, no_show, beta):
  return (I * d) + tardiness - (N * (1 - no_show) * beta)  # I-1?


def calcWaitingtime(p_min, x, p, limit, I, N):
  w = np.zeros((I+1, N+1, limit+1))
  waitingtime = 0

  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k]
    if x[t] > 1:
      for i in range(1, x[t]+1):
        for k in range(limit+1):
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j]

  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waitingtime += w[t][i][k] * k

  waitingtime /= N
  return waitingtime

##########
## TEST ##
##########

calcWaitingtime(p_min, x, p, limit, I, N)


262.83543452018176

In [35]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess


##########
## TEST ##
##########

# calcFracExcess(p_min, I)


<div>
<img src="https://github.com/witusj/obp/blob/master/images/fitness2.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/fitness.png?raw=true" width="450">
</div>

In [36]:
def calcFitness(x, beta, precision, limit, v, N, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  tic = time.perf_counter()
  p_min, limit = calculateProbabilities(x, precision, limit, v, I, d)
  toc = time.perf_counter()
  probT = toc-tic

  # Tardiness calcs
  tic = time.perf_counter()
  tardiness = calcTardiness(p_min, limit, I)
  toc = time.perf_counter()
  tardT = toc-tic

  # Idle time calcs new array of given shape and type, filled with zeros.
  tic = time.perf_counter()
  idletime = calcIdletime(I, d, tardiness, N, no_show, beta)
  toc = time.perf_counter()
  idletimeT = toc-tic

  # Waiting time calcs
  tic = time.perf_counter()
  waitingtime = calcWaitingtime(p_min, x, p, limit, I, N)
  toc = time.perf_counter()
  waitingtimeT = toc-tic

  objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

  print(f"Schedule: {x},\nN: {sum(x)},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

  # Collect into a dictionary
  results = {'p_min': p_min, 'waitingTime': waitingtime,
            'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess

  return results['objVal']

##########
## TEST ##
##########


p, precision_limit = calculate_p(beta, probS, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, N, d, no_show)



results = calcFitness(x, beta, precision, limit, v, N, no_show,
                  I, d, eind, alpha_W, alpha_I, alpha_T)
results


Schedule: [2 1 0 2 0 3 0 0 3 4 3 0 0 2 4 2 3 0 0 1 3 0 1 1 3 3 2 0 4 0 0 2 2 3 3 2 4
 2 3 1 2 3 3 1 3 0 4 4],
N: 89,
Objective value: 327.491748797951,
Prob calculation time: 15.749989 sec,
Waiting time (timer): 262.83543452018176 (33.880663 sec),
Idle time (timer): -3.4040416835362066 (0.000003 sec),
Tardiness (timer): 557.5959583164638 (0.000310 sec)



327.491748797951

In [37]:
def createDist(arr, reverse=False):
    minX, maxX = arr.min(), arr.max()
    minY, maxY = 0, 1
    if(reverse):
        minY, maxY = maxY, minY
    # Scale to values between 0, 1 and make total sum equal to 1
    arrScaled = np.interp(arr, (minX, maxX), (minY, maxY))
    arrNorm = [x / arrScaled.sum() for x in arrScaled]
    return(arrNorm)
    

##########
## TEST ##
##########

arr = np.random.choice(10, 5)
dist1 = createDist(arr)
dist2 = createDist(arr, reverse=True)
arr, dist1, sum(dist1), dist2, sum(dist2)


(array([0, 8, 1, 0, 9]),
 [0.0, 0.4444444444444444, 0.05555555555555555, 0.0, 0.5],
 1.0,
 [0.3333333333333333,
  0.037037037037037056,
  0.2962962962962963,
  0.3333333333333333,
  0.0],
 1.0)

<div>
<img src="https://github.com/witusj/obp/blob/master/images/crossover.png?raw=true" width="450">
<img src="https://github.com/witusj/obp/blob/master/images/crossover2.png?raw=true" width="450">
</div>

In [38]:
def crossover(parent1, parent2, nSwaps):
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    I = len(child1)
    
    # Calculate distribution for choosing intervals that will swap patients
    # The distribution is the normalized schedule. Intervals with zero patients will never be selected.
    p1 = createDist(child1)
    
    # Swap patients parent1 to parent 2
    n1 = nSwaps
    while(n1 > 0):
        i = np.random.choice(I, 1, p=p1)
        child1[i] += -1
        child2[i] += 1
        if(child1[i] == 0):
            p1 = createDist(child1) # If the number of patients in interval i reaches zero, this interval should not be selected again.
        n1 -= 1
    
    # Swap patients parent2 to parent 1
    p2 = createDist(child2)
    
    n2 = nSwaps
    while(n2 > 0):
        i = np.random.choice(I, 1, p=p2)
        child2[i] += -1
        child1[i] += 1
        if(child2[i] == 0):
            p2 = createDist(child2)
        n2 -= 1
        
    return(child1, child2)

##########
## TEST ##
##########

testI = 10
parent1 = np.random.choice(5, testI)
parent2 = np.random.choice(5, testI)

print(parent1, parent2)
nSwaps = 3
children = crossover(parent1, parent2, nSwaps)
children


[4 3 4 3 3 0 4 1 1 1] [2 2 0 2 0 0 2 2 3 0]


(array([4, 5, 4, 1, 3, 0, 4, 1, 1, 1]), array([2, 0, 0, 4, 0, 0, 2, 2, 3, 0]))


<div>
    <img src="https://github.com/witusj/obp/blob/master/images/mutation.png?raw=true" width="450">
    <img src="https://github.com/witusj/obp/blob/master/images/mutation1.png?raw=true" width="450">
</div>

In [39]:
def mutate(child, rate):
    if np.random.rand() < rate:
        x = len(child)
        i, j = np.random.choice(x, 2, replace=False).astype(int)
        a, b = child[i], child[j]
        child[i], child[j] = b, a
    return(child)

##########
## TEST ##
##########

child = children[0].copy()
print(child)
child = mutate(child, 1)
print(child)


[4 5 4 1 3 0 4 1 1 1]
[1 5 4 4 3 0 4 1 1 1]


## Classes

#### Suggesties: [get / set methods](https://www.geeksforgeeks.org/getter-and-setter-in-python/) voor parameters / met tests / nuttig voor optimaliseren hyper parameters, bv mbv bayesian stats

Create a data object to store and generate initial data.

In [40]:
class Data:
    def __init__(self, I=10, N=12, d=5, popSize=5, beta=9, precision=0.9999, no_show=0, eind=0, alpha_I=0.2, alpha_T=0.3, alpha_W=0.5, nSwaps=4, mutationRate=0.1):
        self.I = I
        self.N = N
        self.d = d
        self.popSize = popSize
        self.beta = beta
        self.precision = precision
        self.no_show = no_show
        self.probSize = calcExponentialLimit(self.beta*self.N)+1
        self.p, self.precision_limit = self.__calculate_p()
        self.v, self.limit = self.__calculate_v()
        self.eind = eind
        self.alpha_I, self.alpha_T, self.alpha_W = alpha_I, alpha_T, alpha_W
        self.nSwaps, self.mutationRate = nSwaps, mutationRate
    
    def get_initial_data(self): return np.array((self.I, self.N, self.d, self.beta, self.popSize, self.eind, self.alpha_I, self.alpha_T, self.alpha_W, self.nSwaps, self.mutationRate),
                                                dtype={'names': ('I', 'N', 'd', 'beta', 'popSize', 'eind', 'alpha_I', 'alpha_T', 'alpha_W', 'nSwaps', 'mutationRate'),
                                                'formats': ('i4', 'i4', 'i4', 'i4', 'i4', 'i4', 'f8', 'f8', 'f8', 'i4', 'f8')})

    def get_prob_data(self): return np.array((self.precision, self.no_show, self.probSize, self.precision_limit, self.limit),
                                                dtype={'names': ('precision', 'no_show', 'probSize', 'precision_limit', 'limit'),
                                                'formats': ('f8', 'f8', 'i4', 'f8', 'i4')})

    def calculate_p(self):  return calculate_p(self.beta, self.probSize, self.precision)
    __calculate_p = calculate_p
    
    def calculate_v(self): return calculate_v(self.p, self.beta, self.precision, self.precision_limit, self.N, self.d, self.no_show)
    __calculate_v = calculate_v
    

##########
## TEST ##
##########

newDataObject = Data(I, N)
initData = newDataObject.get_initial_data()
initData[['N', 'popSize']]


array((89, 5),
      dtype={'names': ['N', 'popSize'], 'formats': ['<i4', '<i4'], 'offsets': [4, 16], 'itemsize': 60})

<img src="https://github.com/witusj/obp/blob/master/images/population.png?raw=true" width="600">

Create a random individual. An individual is in this case one schedule.

In [41]:
class Schedule:
    def __init__(self, dataObject):
        self.I = dataObject.get_initial_data()['I']  # Number of intervals
        self.N = dataObject.get_initial_data()['N']  # Number of patients
        self.x = np.zeros(self.I).astype(int)# Empty schedule

    def create_random_schedule(self):
        for i in range(self.N):
            i = np.random.choice(self.I, 1)  # Interval ID
            self.x[i] = self.x[i] + 1

    def create_bw_schedule(self):
        self.x = np.ones(self.I).astype(int)
        self.x[0] = 2
        checkSum = np.sum(self.x)
        print(f'checkSum = {checkSum}')
        diff = checkSum - self.N
        print(f'diff = {diff}')
        if diff > 0:
            i = np.random.choice(range(1, self.I), diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] - 1
        if diff < 0:
            i = np.random.choice(range(1, self.I), -diff, replace=False)
            print(f'i = {i}')
            self.x[i] = self.x[i] + 1
    def get_schedule(self): return(self.x)
    def print_schedule(self): print(self.x)


##########
## TEST ##
##########

dataObject = Data(I=I, N=N, d=d)
scheduleObject = Schedule(dataObject)
scheduleObject.print_schedule()
scheduleObject.create_random_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
[3 3 1 2 3 1 3 0 1 2 0 2 1 3 0 1 3 3 3 0 1 3 0 3 2 1 2 3 3 3 2 3 1 2 0 5 4
 2 0 3 1 1 0 5 1 2 0 1]
89
checkSum = 49
diff = -40
i = [18  5 45  8 19 47  6 30 29 14 41 28 20 11  3 17 21 43 44 40 25 39 37 22
 12 38 16 23 27 24 36  7  1 32  2 15 10 34 31 42]
[2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 2
 2 2 2 2 2 2 2 2 2 1 2]
89


Create population of individuals

In [42]:
# OLD
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in self.individuals])
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array([calcFitness(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show,
                                            I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T)['objVal'] for x in newIndividuals])
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


In [65]:
#NEW
class Population:
    global bestRank
    bestRank = []
    def __init__(self, dataObject, individuals=None):
        self.dataObject = dataObject
        self.params = self.dataObject.get_initial_data()
        self.prob_params = self.dataObject.get_prob_data()
        self.p, self.v = self.dataObject.p, self.dataObject.v
        self.individuals = individuals
        if self.individuals == None: self.__create_new_individuals() 
        self.fitnessScores = []

    def create_new_individuals(self):
        self.individuals = []
        for i in range(self.params['popSize']):
            x = Schedule(self.dataObject)
            x.create_random_schedule()
            x = x.get_schedule()
            self.individuals.append(x)
            
    __create_new_individuals = create_new_individuals
    
    def get_individuals(self): return(self.individuals)
    
    def calc_fitness_scores(self):
            beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[['beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
            precision, limit, no_show = self.prob_params[['precision', 'limit', 'no_show']].tolist()
            self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v, N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in self.individuals))
            
            
    def apply_crossover(self):
        newIndividuals = []
        nSwaps = self.params['nSwaps']*1 ## Times 1 because otherwise refers to memorypoint
        mutationRate = self.params['mutationRate']
        self.population_dist = createDist(self.fitnessScores, reverse=True)
        
        for i in range(self.params['popSize']):
            print(f'Crossover {i}')
            r1, r2 = np.random.choice(self.params['popSize'], 2, replace=False, p=self.population_dist)
            parent1, parent2 = self.individuals[r1], self.individuals[r2]
            child1, child2 = crossover(parent1, parent2, nSwaps)
            newIndividuals.append(mutate(child1, mutationRate))
            newIndividuals.append(mutate(child2, mutationRate))
            
        beta, N, I, d, eind, alpha_W, alpha_I, alpha_T = self.params[[
            'beta', 'N', 'I', 'd', 'eind', 'alpha_W', 'alpha_I', 'alpha_T']].tolist()
        precision, limit, no_show = self.prob_params[[
            'precision', 'limit', 'no_show']].tolist()
        self.fitnessScores = np.array(Parallel(n_jobs=6)(delayed(calcFitness)(x, beta=beta, precision=precision, limit=limit, v=self.v,
                                                                              N=N, no_show=no_show, I=I, d=d, eind=eind, alpha_W=alpha_W, alpha_I=alpha_I, alpha_T=alpha_T) for x in newIndividuals))
        tempList = list(range(self.params['popSize']*2))
        zipped = list(zip(*sorted(zip(self.fitnessScores, tempList))))
        self.fitnessScores = np.array(list(zipped[0][:self.params['popSize']]))
        tempList = zipped[1][:self.params['popSize']]
        self.individuals = list(map(newIndividuals.__getitem__, tempList))
        bestRank.append([self.fitnessScores[0], self.individuals[0]])
        print(self.fitnessScores, '\n', self.individuals)


## Tests

In [66]:
##########
## TEST ##
##########

bestRank = []
dataObject = Data()
populationObject1 = Population(dataObject)
populationObject1.fitnessScores
populationObject1.calc_fitness_scores()
print(populationObject1.fitnessScores)


Schedule: [1 1 2 0 0 0 1 2 0 5],
N: 12,
Objective value: 27.976026526799007,
Prob calculation time: 0.143005 sec,
Waiting time (timer): 20.687274116849803 (0.112839 sec),
Idle time (timer): 0.4647789367482176 (0.000002 sec),
Tardiness (timer): 58.46477893674821 (0.000065 sec)

Schedule: [1 0 0 2 1 0 0 1 5 2],
N: 12,
Objective value: 32.42656536666801,
Prob calculation time: 0.142547 sec,
Waiting time (timer): 23.89133506841189 (0.111714 sec),
Idle time (timer): 6.161795664924128 (0.000002 sec),
Tardiness (timer): 64.16179566492413 (0.000066 sec)

Schedule: [2 1 2 1 2 0 3 0 0 1],
N: 12,
Objective value: 33.1521766012038,
Prob calculation time: 0.144763 sec,
Waiting time (timer): 31.562343789326388 (0.112811 sec),
Idle time (timer): -0.05799058691879111 (0.000002 sec),
Tardiness (timer): 57.94200941308121 (0.000069 sec)

Schedule: [2 2 1 0 0 3 0 1 2 1],
N: 12,
Objective value: 31.48706052556892,
Prob calculation time: 0.148221 sec,
Waiting time (timer): 28.231306249169904 (0.117219 sec),

In [67]:
individuals2 = populationObject1.individuals.copy()
populationObject2 = Population(dataObject, individuals=individuals2)
print(populationObject1.individuals, '\n', populationObject2.individuals)

[array([2, 0, 2, 0, 1, 0, 0, 3, 3, 1]), array([1, 1, 2, 0, 0, 0, 1, 2, 0, 5]), array([2, 2, 1, 0, 0, 3, 0, 1, 2, 1]), array([2, 1, 2, 1, 2, 0, 3, 0, 0, 1]), array([1, 0, 0, 2, 1, 0, 0, 1, 5, 2])] 
 [array([2, 0, 2, 0, 1, 0, 0, 3, 3, 1]), array([1, 1, 2, 0, 0, 0, 1, 2, 0, 5]), array([2, 2, 1, 0, 0, 3, 0, 1, 2, 1]), array([2, 1, 2, 1, 2, 0, 3, 0, 0, 1]), array([1, 0, 0, 2, 1, 0, 0, 1, 5, 2])]


In [68]:
populationObject2.calc_fitness_scores()
populationObject2.apply_crossover()
populationObject2.individuals


Schedule: [1 1 2 0 0 0 1 2 0 5],
N: 12,
Objective value: 27.976026526799007,
Prob calculation time: 0.105900 sec,
Waiting time (timer): 20.687274116849803 (0.110305 sec),
Idle time (timer): 0.4647789367482176 (0.000004 sec),
Tardiness (timer): 58.46477893674821 (0.000183 sec)

Schedule: [2 2 1 0 0 3 0 1 2 1],
N: 12,
Objective value: 31.48706052556892,
Prob calculation time: 0.107797 sec,
Waiting time (timer): 28.231306249169904 (0.112320 sec),
Idle time (timer): -0.05718519803205879 (0.000002 sec),
Tardiness (timer): 57.94281480196794 (0.000058 sec)

Schedule: [2 1 2 1 2 0 3 0 0 1],
N: 12,
Objective value: 33.1521766012038,
Prob calculation time: 0.107723 sec,
Waiting time (timer): 31.562343789326388 (0.111390 sec),
Idle time (timer): -0.05799058691879111 (0.000002 sec),
Tardiness (timer): 57.94200941308121 (0.000056 sec)

Schedule: [2 0 2 0 1 0 0 3 3 1],
N: 12,
Objective value: 29.341883004967677,
Prob calculation time: 0.109983 sec,
Waiting time (timer): 23.76121946432517 (0.122643 s

[array([1, 1, 2, 0, 0, 1, 0, 2, 0, 5]),
 array([2, 0, 2, 0, 0, 0, 1, 3, 0, 4]),
 array([1, 1, 1, 0, 1, 1, 0, 3, 3, 1]),
 array([1, 1, 2, 0, 1, 0, 0, 2, 3, 2]),
 array([2, 0, 1, 0, 1, 1, 0, 4, 2, 1])]

In [69]:
# Initial population
bestRank = []
dataObject = Data()
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
individuals = populationZero.individuals.copy()
generations = 3

# Genetic algorithm loop
for i in range(generations):
    print(f'Generation: {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.calc_fitness_scores()
    populationObject.apply_crossover()
    individuals = populationObject.individuals


Schedule: [1 1 1 3 1 2 1 0 0 2],
N: 12,
Objective value: 31.789239183636894,
Prob calculation time: 0.108832 sec,
Waiting time (timer): 28.736691676051304 (0.087547 sec),
Idle time (timer): 0.041786691222483796 (0.000002 sec),
Tardiness (timer): 58.04178669122249 (0.000111 sec)

Schedule: [1 0 1 2 2 0 0 3 1 2],
N: 12,
Objective value: 30.743053004882405,
Prob calculation time: 0.109524 sec,
Waiting time (timer): 24.909381711516456 (0.118835 sec),
Idle time (timer): 1.7767242982483538 (0.000002 sec),
Tardiness (timer): 59.77672429824836 (0.000055 sec)

Schedule: [0 1 3 3 0 2 2 1 0 0],
N: 12,
Objective value: 37.812384234385505,
Prob calculation time: 0.105982 sec,
Waiting time (timer): 35.80322234323544 (0.133782 sec),
Idle time (timer): 5.021546125535565 (0.000002 sec),
Tardiness (timer): 63.02154612553557 (0.000057 sec)

Schedule: [0 2 0 1 2 0 0 2 1 4],
N: 12,
Objective value: 32.356108743334865,
Prob calculation time: 0.107928 sec,
Waiting time (timer): 24.930962533903255 (0.130150 s

In [70]:

bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF


,Score,Schedule
0,28.548416,"[1, 1, 0, 1, 2, 0, 0, 3, 1, 3]"
1,28.322332,"[1, 1, 1, 0, 1, 0, 0, 2, 1, 5]"
2,28.529561,"[1, 2, 0, 0, 1, 0, 0, 2, 1, 5]"


In [71]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 0 0 2 1 0 0 0 0 0 2 1 0 1 1 0 2 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1
 0 0 1 0 0 2 0 0 1 2 0],
N: 24,
Objective value: 19.242516231482288,
Prob calculation time: 1.577335 sec,
Waiting time (timer): 9.918834173114748 (0.405952 sec),
Idle time (timer): 41.45830427039397 (0.000002 sec),
Tardiness (timer): 17.458304270393985 (0.000100 sec)

Schedule: [2 0 1 1 0 0 1 0 1 1 0 2 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1
 0 0 0 0 2 0 0 1 0 1 1],
N: 24,
Objective value: 14.85203222071566,
Prob calculation time: 1.722183 sec,
Waiting time (timer): 8.123499178882737 (0.264638 sec),
Idle time (timer): 35.33772091527095 (0.000002 sec),
Tardiness (timer): 11.337720915270939 (0.000129 sec)

Schedule: [0 0 1 0 1 0 0 1 1 2 1 2 0 0 0 2 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 3 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 1],
N: 24,
Objective value: 15.280735058623613,
Prob calculation time: 1.621401 sec,
Waiting time (timer): 17.514620729632824 (0.369769 sec),
Idle time (timer): 29.79147794461747 (0.00000

In [72]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score', hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='LimeGreen')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [73]:
scheduleObject = Schedule(dataObject)
scheduleObject.create_bw_schedule()
scheduleObject.print_schedule()
print(scheduleObject.get_schedule().sum())
x = scheduleObject.get_schedule()
calcFitness(x, beta, precision, limit, v, N, no_show,
            I, d, eind, alpha_W, alpha_I, alpha_T)


checkSum = 49
diff = 25
i = [13 41 43 23 14  7 40  3 47  9 46 15  5 25 38 10  4 16 24  6 44 26 11 42
 37]
[2 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 1 0 0 0 0 0 1 0 0]
24
Schedule: [2 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 1 0 0 0 0 0 1 0 0],
N: 24,
Objective value: 17.160971560094175,
Prob calculation time: 17.102917 sec,
Waiting time (timer): 16.002952215786713 (0.868032 sec),
Idle time (timer): 33.93298445629915 (0.000003 sec),
Tardiness (timer): 9.932984456299149 (0.000392 sec)



17.160971560094175

In [74]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 10

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 2 2 0 0 2 1 0 1 0 1 0 0 0 0 0 1
 0 1 0 0 0 1 1 0 0 0 1],
N: 24,
Objective value: 12.830638132189986,
Prob calculation time: 1.791297 sec,
Waiting time (timer): 11.561379733477919 (0.255477 sec),
Idle time (timer): 29.676810397998025 (0.000002 sec),
Tardiness (timer): 5.676810397998031 (0.000111 sec)

Schedule: [1 2 1 0 0 0 0 1 3 0 0 0 0 0 0 0 0 0 1 1 2 0 0 0 1 0 0 1 0 1 0 1 0 0 1 2 0
 0 0 0 1 2 0 0 0 1 1 0],
N: 24,
Objective value: 12.436240270655293,
Prob calculation time: 1.840093 sec,
Waiting time (timer): 7.654449148274366 (0.458491 sec),
Idle time (timer): 31.624101018909244 (0.000002 sec),
Tardiness (timer): 7.624101018909244 (0.000110 sec)

Schedule: [1 1 1 0 1 2 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 1 0 0 0 3 0 1 2 0 0 0 1
 0 0 0 1 0 0 1 1 1 1 0],
N: 24,
Objective value: 29.628845244098066,
Prob calculation time: 1.854115 sec,
Waiting time (timer): 19.779853415772706 (0.466766 sec),
Idle time (timer): 52.194839796314966 (0.00

In [75]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='RoyalBlue')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [76]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 20

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind,
                    alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop
score_list = []
for i in range(generations):
    print(f'Generation {i}')
    tic = time.perf_counter()
    populationObject = Population(dataObject, individuals=individuals)
    toc = time.perf_counter()
    objTime = toc - tic
    populationObject.fitnessScores = fitnessScores
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    tic = time.perf_counter()
    populationObject.apply_crossover()
    toc = time.perf_counter()
    crossTime = toc - tic
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()
    print(f'Population building time: : {objTime}\nCrossover time: {crossTime}\n')


Schedule: [0 0 1 0 0 0 1 0 1 0 0 1 2 0 1 0 1 0 1 1 0 1 0 0 0 0 0 1 2 0 2 0 0 0 1 2 0
 1 1 0 0 0 0 0 0 0 1 2],
N: 24,
Objective value: 20.654493906038994,
Prob calculation time: 1.668820 sec,
Waiting time (timer): 10.763992090429147 (0.401609 sec),
Idle time (timer): 43.24816178311221 (0.000002 sec),
Tardiness (timer): 19.24816178311223 (0.000111 sec)

Schedule: [0 1 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 3 1 2 0 0 0 0 0 1 1 0 0 1 0 1
 0 1 0 0 1 1 0 1 0 1 2],
N: 24,
Objective value: 24.945886600543933,
Prob calculation time: 1.827058 sec,
Waiting time (timer): 11.628598312866783 (0.292354 sec),
Idle time (timer): 49.82407879232869 (0.000002 sec),
Tardiness (timer): 25.824078792328695 (0.000114 sec)

Schedule: [0 0 0 1 2 1 1 2 1 0 0 0 0 2 1 0 1 1 2 0 0 0 0 0 2 0 1 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 1 0 0],
N: 24,
Objective value: 17.16409410042492,
Prob calculation time: 1.708188 sec,
Waiting time (timer): 27.182697295669552 (0.396832 sec),
Idle time (timer): 26.485025303595165 (0.000

In [77]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='Tomato')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [78]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 10
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [1 1 0 0 0 1 0 0 2 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1 2 1 0 1 1 0 0 0 0 0 1 0 1
 0 0 0 1 0 1 1 0 0 1 1],
N: 24,
Objective value: 13.203267316081751,
Prob calculation time: 1.654155 sec,
Waiting time (timer): 6.4303101581094415 (0.159062 sec),
Idle time (timer): 33.7185720880633 (0.000002 sec),
Tardiness (timer): 9.718572088063285 (0.000109 sec)

Schedule: [0 0 0 1 2 0 3 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 2 0 1 1 0 0 1 0 1
 1 0 0 0 1 0 0 1 0 0 0],
N: 24,
Objective value: 13.110374611094421,
Prob calculation time: 1.625735 sec,
Waiting time (timer): 16.965736423783188 (0.269093 sec),
Idle time (timer): 26.540133402635234 (0.000004 sec),
Tardiness (timer): 2.5401334026352456 (0.000171 sec)

Schedule: [0 1 0 0 0 1 0 2 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 2 0 1 1 2 0 1 0 0 0 0 1 1
 0 1 0 0 1 0 1 0 0 1 1],
N: 24,
Objective value: 19.26217613429587,
Prob calculation time: 1.665081 sec,
Waiting time (timer): 11.95137017028709 (0.236350 sec),
Idle time (timer): 40.136046776968385 (0.0000

In [79]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkOrange')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")


In [80]:
bestRank = []
precision = 0.9999
N = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
eind = 0
alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4
popSize = 15
generations = 5
nSwaps = 6

# Initial population
dataObject = Data(N=N, beta=beta, d=d, I=I, no_show=no_show, eind=eind, alpha_I=alpha_I, alpha_T=alpha_T, alpha_W=alpha_W, popSize=popSize, nSwaps=nSwaps)
populationZero = Population(dataObject)
populationZero.calc_fitness_scores()
best_score = min(populationZero.fitnessScores)
score_list = [best_score]
individuals = populationZero.individuals.copy()
fitnessScores = populationZero.fitnessScores.copy()

# Genetic algorithm loop

for i in range(generations):
    print(f'Generation {i}')
    populationObject = Population(dataObject, individuals=individuals)
    populationObject.fitnessScores = fitnessScores
    populationObject.apply_crossover()
    best_score = min(populationObject.fitnessScores)
    score_list.append(best_score)
    individuals = populationObject.individuals.copy()
    fitnessScores = populationObject.fitnessScores.copy()


Schedule: [0 0 1 1 0 0 0 0 0 1 0 2 1 0 0 0 0 0 0 0 1 0 2 1 0 0 0 0 0 1 2 2 1 1 0 0 1
 0 0 0 0 1 1 0 1 0 1 2],
N: 24,
Objective value: 29.038713013780885,
Prob calculation time: 1.601887 sec,
Waiting time (timer): 13.046707360641925 (0.380055 sec),
Idle time (timer): 55.70005011587352 (0.000002 sec),
Tardiness (timer): 31.700050115873534 (0.000110 sec)

Schedule: [3 0 0 0 1 1 0 0 2 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 2 0 1 1 1 2 0 2 1 0
 0 1 0 0 0 1 0 1 0 1 0],
N: 24,
Objective value: 23.898261186464957,
Prob calculation time: 1.610234 sec,
Waiting time (timer): 16.088854894942546 (0.416544 sec),
Idle time (timer): 45.10453204747989 (0.000002 sec),
Tardiness (timer): 21.104532047479896 (0.000124 sec)

Schedule: [2 0 0 1 3 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 2 0 0 0 0 0 2
 2 0 0 0 0 1 1 1 0 2 1],
N: 24,
Objective value: 27.226515065148092,
Prob calculation time: 1.590123 sec,
Waiting time (timer): 10.355490787599157 (0.499149 sec),
Idle time (timer): 54.47386458351406 (0.00

In [81]:
bestRankDF = pd.DataFrame(bestRank, columns=['Score', 'Schedule'])
bestRank
bestRankDF['Generation'] = bestRankDF.index.values
fig = px.line(bestRankDF, x='Generation', y='Score',
              hover_data=['Schedule'], markers=True)
fig.update_traces(line_color='DarkCyan')
fig.show()
fileName = f'p{popSize}g{generations}n{nSwaps}.html'
fig.write_html(f'charts/{fileName}', include_plotlyjs="cdn")
